In [2]:
import pandas as pd
import numpy as np
import requests
import keys
import json
from tqdm import tqdm

In [2]:
df = pd.read_csv('tm_events.csv')

In [3]:
venues = list(set(df.venue_name))

In [4]:
venues = [x.strip() for x in venues]

In [6]:
url = 'https://api.songkick.com/api/3.0/search/venues.json?'
success = []
fail = []
frame_list = []
for venue in tqdm(venues):
    try:
        req = requests.get(f'{url}query={venue}&apikey={keys.songkick}')
        x = json.loads(req.text)
        df = pd.DataFrame(x['resultsPage']['results']['venue'])
        frame_list.append(df)
        success.append(venue)
    except:
        fail.append(venue)
        


100%|██████████| 2779/2779 [20:22<00:00,  2.38it/s]  


In [19]:
venues[2]

'The Roxy'

In [62]:
df = pd.concat(frame_list)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [63]:
df.reset_index(inplace=True)

In [64]:
df = df[['displayName','id','capacity', 'street', 'city', 'zip', 'lat','lng','metroArea' ]]

In [65]:
df.rename(columns = {'displayName': 'venue_name', 'lat':'latitude', 'lng':'longitude'},inplace=True)

In [67]:
df.reset_index(inplace=True)
df.rename(columns ={'index' : 'i'}, inplace=True)
df['metro_area'] = df.metroArea.apply(lambda x: x['displayName'])
df['country'] = df.metroArea.apply(lambda x: x['country']['displayName'])
df.drop(columns = ['metroArea', 'i'],inplace=True)

In [68]:
us_venues = df[df.country=='US']

In [69]:
us_venues.city[0]

{'country': {'displayName': 'US'},
 'state': {'displayName': 'CA'},
 'displayName': 'Oakland'}

In [70]:
us_venues['state'] = us_venues.city.apply(lambda x: x['state']['displayName'])
us_venues['city_name'] = us_venues.city.apply(lambda x: x['displayName'])

KeyError: 'state'

In [61]:
us_venues.to_csv('sk_venues.csv')

In [4]:
df = pd.read_csv('sk_venues.csv', dtype='object')

In [5]:
df.drop(columns = ['Unnamed: 0'], inplace=True)

In [6]:
venue_id = df[['venue_name', 'id']]

In [7]:
ids = list(set(venue_id.id))

In [9]:
url = 'https://api.songkick.com/api/3.0/venues/'
success = []
fail = []
frame_list = []
for id_ in tqdm(ids):
    try:
        req = requests.get(f'{url}{str(id_)}/calendar.json?apikey={keys.songkick}')
        x = json.loads(req.text)
        event_df = pd.DataFrame(x['resultsPage']['results']['event'])
        frame_list.append(event_df)
        success.append(id_)
    except:
        fail.append(id_)
        

100%|██████████| 8421/8421 [1:36:07<00:00,  1.51it/s]  


In [22]:
frames = frame_list.copy()

In [23]:
df = pd.concat(frames)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [12]:
df.head()

,ageRestriction,displayName,end,flaggedAsEnded,id,location,performance,popularity,series,start,status,type,uri,venue
0,None,Jose Negroni and Negroni's Trio at Miami Dade ...,NaN,False,38048224,"{'city': 'Miami, FL, US', 'lat': 25.77248, 'ln...","[{'id': 72226849, 'displayName': 'Jose Negroni...",0.000066,NaN,"{'date': '2019-08-23', 'datetime': '2019-08-23...",ok,Concert,http://www.songkick.com/concerts/38048224-jose...,"{'id': 2868, 'displayName': 'Miami Dade County..."
1,None,Jose Negroni and Negroni's Trio at Miami Dade ...,NaN,False,38049129,"{'city': 'Miami, FL, US', 'lat': 25.77248, 'ln...","[{'id': 72228354, 'displayName': 'Jose Negroni...",0.000066,NaN,"{'date': '2019-09-29', 'datetime': '2019-09-29...",ok,Concert,http://www.songkick.com/concerts/38049129-jose...,"{'id': 2868, 'displayName': 'Miami Dade County..."
2,None,Benise at Miami Dade County Auditorium (Octobe...,NaN,False,37362664,"{'city': 'Miami, FL, US', 'lat': 25.77248, 'ln...","[{'id': 71018739, 'displayName': 'Benise', 'bi...",0.000643,NaN,"{'date': '2019-10-12', 'datetime': '2019-10-12...",ok,Concert,http://www.songkick.com/concerts/37362664-beni...,"{'id': 2868, 'displayName': 'Miami Dade County..."
3,None,Enrique Chia at Miami Dade County Auditorium (...,NaN,False,38998599,"{'city': 'Miami, FL, US', 'lat': 25.77248, 'ln...","[{'id': 73868640, 'displayName': 'Enrique Chia...",0.000194,NaN,"{'date': '2019-10-13', 'datetime': '2019-10-13...",ok,Concert,http://www.songkick.com/concerts/38998599-enri...,"{'id': 2868, 'displayName': 'Miami Dade County..."
4,None,Meme Solis at Miami Dade County Auditorium (Oc...,NaN,False,38971387,"{'city': 'Miami, FL, US', 'lat': 25.77248, 'ln...","[{'id': 73821386, 'displayName': 'Meme Solis',...",0.000006,NaN,"{'date': '2019-10-19', 'datetime': '2019-10-19...",ok,Concert,http://www.songkick.com/concerts/38971387-meme...,"{'id': 2868, 'displayName': 'Miami Dade County..."


In [55]:
# df['location'] = df.location.apply(lambda x: ast.literal_eval(x))
# df['venue'] = df.venue.apply(lambda x: ast.literal_eval(x))
# df['performance'] = df.performance.apply(lambda x: ast.literal_eval(x))
# df['start'] = df.start.apply(lambda x: ast.literal_eval(x))

In [24]:
def try_artist(x, i):
    try:
        return x[i]['displayName']
    except:
        return np.nan
    
df.reset_index(inplace=True)
df.rename(columns={'index' : 'i'},inplace=True)
df['venue_id'] = df.venue.apply(lambda x: x['id'])
df['headliner'] = df.performance.apply(lambda x: try_artist(x, 0))
df['support'] = df.performance.apply(lambda x: try_artist(x, 1))
df['event_time'] = df.start.apply(lambda x: pd.to_datetime(x['datetime']))
df.drop(columns = ['flaggedAsEnded','series', 'end','status','uri', 'i','ageRestriction','venue', 'performance', 'location', 'displayName', 'start'], inplace=True)

In [30]:
df.rename(columns={'popularity':'popularity_sk'},inplace=True)

In [31]:
df.to_csv('sk_events.csv')

In [2]:
venue_df = pd.read_csv('sk_venues.csv')

In [4]:
events_df = pd.read_csv('sk_events.csv')

In [6]:
tm_df = pd.read_csv('tm_events.csv')

In [7]:
artists = list(set(events_df.headliner))

In [8]:
artists = [x for x in artists if x != 'nan']

In [13]:
frame_list = []

In [14]:

url = 'https://api.songkick.com/api/3.0/search/artists.json?'
success = []
fail = []
for artist in tqdm(artists):
    try:
        req = requests.get(f'{url}apikey={keys.songkick}&query={artist}')
        x = json.loads(req.text)
        artist_df = pd.DataFrame(x['resultsPage']['results']['artist'])
        frame_list.append(artist_df)
        success.append(artist)
    except:
        fail.append(artist)

100%|██████████| 11190/11190 [1:26:16<00:00,  2.24it/s]  


In [15]:
frames = frame_list.copy()

In [21]:
df = pd.concat(frames[1:])

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [25]:
df.drop(columns=['identifier','uri'],inplace=True)

In [30]:
df.drop_duplicates(inplace=True)

In [34]:
df.to_csv('artists.csv')

In [44]:
df.head()

,displayName,id,onTourUntil
0,MGMT,137345,2019-11-22
1,T.I.,17879,None
2,P!NK,9862999,2019-09-27
3,M.I.A.,493293,None
4,AWOLNATION,2489666,None


In [53]:
req = requests.get(f'https://api.songkick.com/api/3.0/artists/137345/gigography.json?apikey={keys.songkick}')
x = json.loads(req.text)

In [60]:

url = 'https://api.songkick.com/api/3.0/'
frame_list = []
success = []
fail = []
for id_ in tqdm(ids):
    try:
        req = requests.get(f'{url}artists/{id_}/gigography.json?apikey={keys.songkick}')
        x = json.loads(req.text)
        gig_df = pd.DataFrame(x['resultsPage']['results']['event'])
        frame_list.append(gig_df)
        success.append(id_)
    except:
        fail.append(id_)

100%|██████████| 50839/50839 [6:39:02<00:00,  2.23it/s]    


In [62]:
df = pd.concat(frame_list)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [64]:
df.shape

(897058, 14)

In [66]:
df.to_csv('gigography.csv')

In [2]:
df = pd.read_csv('gigography.csv')

In [3]:
df.shape

(897058, 15)